In [11]:
import pandas as pd
import os
import sqlite3 as sql3

In [12]:
currentdir = os.getcwd()
db_path = f"{currentdir}\\S30 ETL Assignment.db"
print(db_path)

c:\Users\vic_d\Documents\.Career\jobhungint_2024\exam_eastvantage\S30 ETL Assignment.db


In [13]:
def tables_extract(db_path):
    conn = sql3.connect(db_path)
    dfs = {}

    query = "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';"
    tables = pd.read_sql(query, conn)['name'].tolist()

    for table in tables:
        dfs[table] = pd.read_sql(f"SELECT * FROM {table}", conn)

    conn.close()
    
    return dfs

In [14]:
dfs = tables_extract(db_path)

In [15]:
# Create filtered customer_sales subset
df_customers = dfs['customers'][(dfs['customers']['age'] >= 18) & (dfs['customers']['age'] <= 35)]
df_customer_sales = df_customers.merge(dfs['sales'], on='customer_id', how='inner')

# Create orders_items subset
df_orders_items = dfs['orders'].merge(dfs['items'], on='item_id', how='inner')

# Process df_orders_items
df_orders_items = df_orders_items[df_orders_items['quantity'] > 0]
df_orders_items['quantity'] = df_orders_items['quantity'].astype('int64')

# Merge customer_sales and df_orders_items ON sales_id
df_merged_all = df_customer_sales.merge(df_orders_items, on='sales_id', how='inner')

# Aggregate
df_final = (df_merged_all.groupby(['customer_id', 'age', 'item_name'])
                    .agg(order_quantity=('quantity', 'sum'))
                    .reset_index()
                    .query('order_quantity > 0')
                    .sort_values(by=['customer_id', 'order_quantity'], ascending=[True, False]))

df_final.to_csv('test_pandas.csv',sep=';',index=False)
